In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from plaid.datasets import CATHStructureDataModule

shard_dir = "/data/lux70/data/cath/shards/"
pdb_dir = "/data/bucket/lux70/data/cath/dompdb"
# shard_dir = "/data/lux70/data/rocklin/shards/"
# pdb_dir = "/data/bucket/lux70/data/rocklin/structures/"

max_seq_len=256
dm = CATHStructureDataModule(
    shard_dir,
    pdb_dir,
    seq_len=max_seq_len,
    batch_size=32,
    max_num_samples=32,
    shuffle_val_dataset=False
) 
    
dm.setup()
val_dataloader = dm.val_dataloader()
batch = next(iter(val_dataloader))
print(len(val_dataloader.dataset))

32


In [6]:
from cheap.proteins import LatentToSequence, LatentToStructure
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
l2seq = LatentToSequence()
l2struct = LatentToStructure()
l2struct.to(device)

loading esmfold model...
ESMFold model created in 46.14 seconds.


In [7]:
x, sequences = batch
headers = [seq_to_header[s] for s in sequences]
seqs = [s[:64] for s in sequences]

ValueError: too many values to unpack (expected 2)

In [ ]:
pdbstrs, raw_outputs, metrics = l2struct.to_structure(x.to(torch.float32), seqs, num_recycles=4, return_metrics=True)

In [ ]:
metrics

In [15]:
for i in range(10):
    with open("pred_from_latent_{}.pdb".format(i), "w") as f:
        f.write(pdbstrs[i])

In [ ]:
# del l2struct

In [16]:
from plaid.esmfold import esmfold_v1
efold = esmfold_v1()

In [17]:
efold.to(device)
with torch.no_grad():
    pdbstr_inferred = efold.infer_pdbs(seqs)

In [18]:
for i in range(10):
    with open("pred_from_seq_{}.pdb".format(i), "w") as f:
        f.write(pdbstr_inferred[i])

In [20]:
import os
import shutil
for i, name in enumerate(cath_ids[:10]):
    original = "/shared/amyxlu/data/cath/full/dompdb/" + name
    shutil.copy(original, f"{i}_name.pdb")
    

In [19]:
from plaid.evaluation import run_tmalign
scores = []
for i in range(10):
    tmscore = run_tmalign(f"pred_from_latent_{i}.pdb", f"pred_from_seq_{i}.pdb")
    scores.append(tmscore)

scores

[0.3898,
 0.89816,
 0.4072,
 0.33646,
 0.82221,
 0.26164,
 0.20608,
 0.36169,
 0.78232,
 0.60658]

In [21]:
from plaid.evaluation import run_tmalign
scores2 = []
for i in range(10):
    tmscore = run_tmalign(f"pred_from_latent_{i}.pdb", f"{i}_name.pdb")
    scores2.append(tmscore)
print(scores2)

[0.42449, 0.49204, 0.26532, 0.13074, 0.46336, 0.22204, 0.20049, 0.33428, 0.33495, 0.22423]


In [22]:
from plaid.evaluation import run_tmalign
scores3 = []
for i in range(10):
    tmscore = run_tmalign(f"pred_from_seq_{i}.pdb", f"{i}_name.pdb")
    scores3.append(tmscore)
print(scores3)

[0.21496, 0.49122, 0.15789, 0.13635, 0.4812, 0.20423, 0.21574, 0.45479, 0.37716, 0.22346]
